In [62]:
# importing the Dataset

import numpy as np
import pandas as pd
from tqdm import tqdm
import re
import nltk
from nltk import sent_tokenize
from gensim.utils import simple_preprocess
import gensim
from sklearn.metrics import accuracy_score,classification_report

messages = pd.read_csv('/kaggle/input/smsspamcollection/SMSSpamCollection.txt', sep='\t',
                           names=["label", "message"])

In [4]:
messages

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [5]:
messages['message'].loc[451]

'hanks lotsly!'

In [6]:
#Data cleaning and preprocessing
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [8]:
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [9]:
corpus

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl week word back like fun still tb ok xxx std chg send rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press copi friend callertun',
 'winner valu network custom select receivea prize reward claim call claim code kl valid hour',
 'mobil month u r entitl updat latest colour mobil camera free call mobil updat co free',
 'gonna home soon want talk stuff anymor tonight k cri enough today',
 'six chanc win cash pound txt csh send cost p day day tsandc appli repli hl info',
 'urgent week free membership prize jackpot txt word claim c www dbuk net lccltd pobox ldnw rw',
 'search right word thank breather

In [10]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500) ##, binary=True, ngram_range=(2,2))
X = cv.fit_transform(corpus).toarray()

In [11]:
X.shape

(5572, 2500)

In [12]:
y=pd.get_dummies(messages['label'])
y=y.iloc[:,1].values

In [13]:
y

array([False, False,  True, ..., False, False, False])

In [14]:
# Train Test Split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)


In [15]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

In [16]:
#prediction
y_pred=spam_detect_model.predict(X_test)

In [18]:
score=accuracy_score(y_test,y_pred)
print(score)

0.9856502242152466


In [19]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

       False       0.99      0.99      0.99       953
        True       0.96      0.94      0.95       162

    accuracy                           0.99      1115
   macro avg       0.97      0.97      0.97      1115
weighted avg       0.99      0.99      0.99      1115



In [20]:
# Creating the TFIDF model
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features=2500)
X = tv.fit_transform(corpus).toarray()

In [21]:
# Train Test Split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)


In [22]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

In [23]:
#prediction
y_pred=spam_detect_model.predict(X_test)

In [24]:
score=accuracy_score(y_test,y_pred)
print(score)

0.979372197309417


In [25]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

       False       1.00      0.98      0.99       978
        True       0.86      1.00      0.92       137

    accuracy                           0.98      1115
   macro avg       0.93      0.99      0.96      1115
weighted avg       0.98      0.98      0.98      1115



## Word2vec Implementation

In [26]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 39.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.0 which is incompatible.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, but you have scikit-learn 1.2.2 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.2 which is incompat

In [ ]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

[========================--------------------------] 48.0% 798.9/1662.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=10000.0 (msgs/sec)
NotebookApp.rate_limit_window=1.0 (secs)



[============================----------------------] 56.0% 931.8/1662.8MB downloaded

In [8]:
vec_king = wv['king']

In [9]:
vec_king

array([ 1.25976562e-01,  2.97851562e-02,  8.60595703e-03,  1.39648438e-01,
       -2.56347656e-02, -3.61328125e-02,  1.11816406e-01, -1.98242188e-01,
        5.12695312e-02,  3.63281250e-01, -2.42187500e-01, -3.02734375e-01,
       -1.77734375e-01, -2.49023438e-02, -1.67968750e-01, -1.69921875e-01,
        3.46679688e-02,  5.21850586e-03,  4.63867188e-02,  1.28906250e-01,
        1.36718750e-01,  1.12792969e-01,  5.95703125e-02,  1.36718750e-01,
        1.01074219e-01, -1.76757812e-01, -2.51953125e-01,  5.98144531e-02,
        3.41796875e-01, -3.11279297e-02,  1.04492188e-01,  6.17675781e-02,
        1.24511719e-01,  4.00390625e-01, -3.22265625e-01,  8.39843750e-02,
        3.90625000e-02,  5.85937500e-03,  7.03125000e-02,  1.72851562e-01,
        1.38671875e-01, -2.31445312e-01,  2.83203125e-01,  1.42578125e-01,
        3.41796875e-01, -2.39257812e-02, -1.09863281e-01,  3.32031250e-02,
       -5.46875000e-02,  1.53198242e-02, -1.62109375e-01,  1.58203125e-01,
       -2.59765625e-01,  

In [10]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [11]:
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower()
    review = review.split()
    
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

NameError: name 're' is not defined

In [13]:
len(corpus)

0

In [70]:
# words=[]
# for sent in corpus:
#     sent_token=sent_tokenize(sent)
#     for sent in sent_token:
#         words.append(simple_preprocess(sent))

In [14]:
words = []
for doc in corpus:
    sents = sent_tokenize(doc)
    if not sents:
        words.append([])                # placeholder for empty/invalid doc
    else:
        for sent in sents:
            words.append(simple_preprocess(sent))

In [15]:
len(words)

0

In [38]:
### Lets train Word2vec from scratch
model=gensim.models.Word2Vec(words,window=5,min_count=2)

model.wv.index_to_key

NameError: name 'gensim' is not defined

In [39]:
model.wv.index_to_key

NameError: name 'model' is not defined

In [37]:
model.corpus_count

NameError: name 'model' is not defined

In [77]:
model.epochs

5

In [78]:
model.wv.similar_by_word('kid')

[('enjoy', 0.9965601563453674),
 ('work', 0.9964696168899536),
 ('happy', 0.9963882565498352),
 ('life', 0.9963331818580627),
 ('person', 0.9962465167045593),
 ('amp', 0.9962289929389954),
 ('hello', 0.9962262511253357),
 ('great', 0.9962126016616821),
 ('thing', 0.9962121248245239),
 ('haha', 0.9961975812911987)]

In [79]:
model.wv['kid'].shape

(100,)

In [35]:
def avg_word2vec(doc):

    # remove out-of-vocabulary words
    #sent = [word for word in doc if word in model.wv.index_to_key]
    #print(sent)

    
    # Collect only in-vocabulary vectors
    vecs = [model.wv[word] for word in doc if word in model.wv.index_to_key]
    
    if not vecs:
        # No known words → return zero-vector of correct size
        return np.zeros(model.vector_size, dtype=float)
    
    # Otherwise, compute the mean normally
    return np.mean(vecs, axis=0)    

In [ ]:
# !pip install tqdm

In [82]:
words[73]

['performed']

In [83]:
type(model.wv.index_to_key)

list

In [84]:

#apply for the entire sentences
X=[]
for i in tqdm(range(len(words))):
    print("Hello",i)
    X.append(avg_word2vec(words[i]))

 10%|▉         | 553/5572 [00:00<00:00, 5523.70it/s]

Hello 0
Hello 1
Hello 2
Hello 3
Hello 4
Hello 5
Hello 6
Hello 7
Hello 8
Hello 9
Hello 10
Hello 11
Hello 12
Hello 13
Hello 14
Hello 15
Hello 16
Hello 17
Hello 18
Hello 19
Hello 20
Hello 21
Hello 22
Hello 23
Hello 24
Hello 25
Hello 26
Hello 27
Hello 28
Hello 29
Hello 30
Hello 31
Hello 32
Hello 33
Hello 34
Hello 35
Hello 36
Hello 37
Hello 38
Hello 39
Hello 40
Hello 41
Hello 42
Hello 43
Hello 44
Hello 45
Hello 46
Hello 47
Hello 48
Hello 49
Hello 50
Hello 51
Hello 52
Hello 53
Hello 54
Hello 55
Hello 56
Hello 57
Hello 58
Hello 59
Hello 60
Hello 61
Hello 62
Hello 63
Hello 64
Hello 65
Hello 66
Hello 67
Hello 68
Hello 69
Hello 70
Hello 71
Hello 72
Hello 73
Hello 74
Hello 75
Hello 76
Hello 77
Hello 78
Hello 79
Hello 80
Hello 81
Hello 82
Hello 83
Hello 84
Hello 85
Hello 86
Hello 87
Hello 88
Hello 89
Hello 90
Hello 91
Hello 92
Hello 93
Hello 94
Hello 95
Hello 96
Hello 97
Hello 98
Hello 99
Hello 100
Hello 101
Hello 102
Hello 103
Hello 104
Hello 105
Hello 106
Hello 107
Hello 108
Hello 109
Hello 110


 40%|███▉      | 2226/5572 [00:00<00:00, 5546.91it/s]

Hello 1105
Hello 1106
Hello 1107
Hello 1108
Hello 1109
Hello 1110
Hello 1111
Hello 1112
Hello 1113
Hello 1114
Hello 1115
Hello 1116
Hello 1117
Hello 1118
Hello 1119
Hello 1120
Hello 1121
Hello 1122
Hello 1123
Hello 1124
Hello 1125
Hello 1126
Hello 1127
Hello 1128
Hello 1129
Hello 1130
Hello 1131
Hello 1132
Hello 1133
Hello 1134
Hello 1135
Hello 1136
Hello 1137
Hello 1138
Hello 1139
Hello 1140
Hello 1141
Hello 1142
Hello 1143
Hello 1144
Hello 1145
Hello 1146
Hello 1147
Hello 1148
Hello 1149
Hello 1150
Hello 1151
Hello 1152
Hello 1153
Hello 1154
Hello 1155
Hello 1156
Hello 1157
Hello 1158
Hello 1159
Hello 1160
Hello 1161
Hello 1162
Hello 1163
Hello 1164
Hello 1165
Hello 1166
Hello 1167
Hello 1168
Hello 1169
Hello 1170
Hello 1171
Hello 1172
Hello 1173
Hello 1174
Hello 1175
Hello 1176
Hello 1177
Hello 1178
Hello 1179
Hello 1180
Hello 1181
Hello 1182
Hello 1183
Hello 1184
Hello 1185
Hello 1186
Hello 1187
Hello 1188
Hello 1189
Hello 1190
Hello 1191
Hello 1192
Hello 1193
Hello 1194
Hello 1195

 60%|██████    | 3355/5572 [00:00<00:00, 5596.70it/s]

Hello 2226
Hello 2227
Hello 2228
Hello 2229
Hello 2230
Hello 2231
Hello 2232
Hello 2233
Hello 2234
Hello 2235
Hello 2236
Hello 2237
Hello 2238
Hello 2239
Hello 2240
Hello 2241
Hello 2242
Hello 2243
Hello 2244
Hello 2245
Hello 2246
Hello 2247
Hello 2248
Hello 2249
Hello 2250
Hello 2251
Hello 2252
Hello 2253
Hello 2254
Hello 2255
Hello 2256
Hello 2257
Hello 2258
Hello 2259
Hello 2260
Hello 2261
Hello 2262
Hello 2263
Hello 2264
Hello 2265
Hello 2266
Hello 2267
Hello 2268
Hello 2269
Hello 2270
Hello 2271
Hello 2272
Hello 2273
Hello 2274
Hello 2275
Hello 2276
Hello 2277
Hello 2278
Hello 2279
Hello 2280
Hello 2281
Hello 2282
Hello 2283
Hello 2284
Hello 2285
Hello 2286
Hello 2287
Hello 2288
Hello 2289
Hello 2290
Hello 2291
Hello 2292
Hello 2293
Hello 2294
Hello 2295
Hello 2296
Hello 2297
Hello 2298
Hello 2299
Hello 2300
Hello 2301
Hello 2302
Hello 2303
Hello 2304
Hello 2305
Hello 2306
Hello 2307
Hello 2308
Hello 2309
Hello 2310
Hello 2311
Hello 2312
Hello 2313
Hello 2314
Hello 2315
Hello 2316

 81%|████████▏ | 4539/5572 [00:00<00:00, 5688.04it/s]

Hello 3396
Hello 3397
Hello 3398
Hello 3399
Hello 3400
Hello 3401
Hello 3402
Hello 3403
Hello 3404
Hello 3405
Hello 3406
Hello 3407
Hello 3408
Hello 3409
Hello 3410
Hello 3411
Hello 3412
Hello 3413
Hello 3414
Hello 3415
Hello 3416
Hello 3417
Hello 3418
Hello 3419
Hello 3420
Hello 3421
Hello 3422
Hello 3423
Hello 3424
Hello 3425
Hello 3426
Hello 3427
Hello 3428
Hello 3429
Hello 3430
Hello 3431
Hello 3432
Hello 3433
Hello 3434
Hello 3435
Hello 3436
Hello 3437
Hello 3438
Hello 3439
Hello 3440
Hello 3441
Hello 3442
Hello 3443
Hello 3444
Hello 3445
Hello 3446
Hello 3447
Hello 3448
Hello 3449
Hello 3450
Hello 3451
Hello 3452
Hello 3453
Hello 3454
Hello 3455
Hello 3456
Hello 3457
Hello 3458
Hello 3459
Hello 3460
Hello 3461
Hello 3462
Hello 3463
Hello 3464
Hello 3465
Hello 3466
Hello 3467
Hello 3468
Hello 3469
Hello 3470
Hello 3471
Hello 3472
Hello 3473
Hello 3474
Hello 3475
Hello 3476
Hello 3477
Hello 3478
Hello 3479
Hello 3480
Hello 3481
Hello 3482
Hello 3483
Hello 3484
Hello 3485
Hello 3486

100%|██████████| 5572/5572 [00:00<00:00, 5647.08it/s]

Hello 4565
Hello 4566
Hello 4567
Hello 4568
Hello 4569
Hello 4570
Hello 4571
Hello 4572
Hello 4573
Hello 4574
Hello 4575
Hello 4576
Hello 4577
Hello 4578
Hello 4579
Hello 4580
Hello 4581
Hello 4582
Hello 4583
Hello 4584
Hello 4585
Hello 4586
Hello 4587
Hello 4588
Hello 4589
Hello 4590
Hello 4591
Hello 4592
Hello 4593
Hello 4594
Hello 4595
Hello 4596
Hello 4597
Hello 4598
Hello 4599
Hello 4600
Hello 4601
Hello 4602
Hello 4603
Hello 4604
Hello 4605
Hello 4606
Hello 4607
Hello 4608
Hello 4609
Hello 4610
Hello 4611
Hello 4612
Hello 4613
Hello 4614
Hello 4615
Hello 4616
Hello 4617
Hello 4618
Hello 4619
Hello 4620
Hello 4621
Hello 4622
Hello 4623
Hello 4624
Hello 4625
Hello 4626
Hello 4627
Hello 4628
Hello 4629
Hello 4630
Hello 4631
Hello 4632
Hello 4633
Hello 4634
Hello 4635
Hello 4636
Hello 4637
Hello 4638
Hello 4639
Hello 4640
Hello 4641
Hello 4642
Hello 4643
Hello 4644
Hello 4645
Hello 4646
Hello 4647
Hello 4648
Hello 4649
Hello 4650
Hello 4651
Hello 4652
Hello 4653
Hello 4654
Hello 4655

In [85]:
type(X)

list

In [86]:
X[0]

array([-0.07348721,  0.24333604,  0.02522965,  0.05252615,  0.0845446 ,
       -0.34037998,  0.10537364,  0.53934234, -0.23115861, -0.16840571,
       -0.11665764, -0.31422147,  0.0146966 ,  0.11085427,  0.095819  ,
       -0.22682196,  0.01124572, -0.33123484,  0.02334019, -0.52597183,
        0.15335265,  0.13727795,  0.10184411, -0.13196482, -0.17468841,
        0.01792875, -0.17146604, -0.14894323, -0.26405525,  0.00420251,
        0.30537084,  0.02337169,  0.0464271 , -0.1359442 , -0.1345272 ,
        0.24821179,  0.0719082 , -0.20173918, -0.17702825, -0.47558644,
       -0.00359754, -0.2397645 , -0.05162602,  0.01338076,  0.21281894,
       -0.02623428, -0.18517423, -0.05397284,  0.07682566,  0.17697603,
        0.118655  , -0.20245112, -0.03218474, -0.02426144, -0.12045481,
        0.1106784 ,  0.18463147, -0.01678273, -0.31617662,  0.09084473,
        0.07479045,  0.20837897, -0.13773169, -0.0360205 , -0.3065317 ,
        0.14126717,  0.09216056,  0.15595725, -0.33399153,  0.31

In [87]:
X[0].shape

(100,)

In [88]:
X_new = (np.array(X, dtype= object))

In [89]:
X_new[0].shape

(100,)

In [90]:
X_new.shape

(5572, 100)

In [91]:
# Train Test Split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size = 0.20, random_state = 0)

In [92]:
X_train.shape, y_train.shape

((4457, 100), (4457,))

In [93]:
X_train[0].shape, X_train[1].shape

((100,), (100,))

In [94]:
type(X_train), type(X_train[0])

(numpy.ndarray, numpy.ndarray)

In [95]:
X_mat = np.stack(X_train)    # shape (n_samples, f)
print(X_mat.shape, X_mat.dtype)

(4457, 100) object


In [96]:
from sklearn.ensemble import RandomForestClassifier
spam_detect_model = RandomForestClassifier(
    n_estimators=100, 
    max_depth=None,       
    random_state=42,      
    n_jobs=-1             
).fit(X_train, y_train)

In [97]:
#prediction
y_pred=spam_detect_model.predict(X_test)

In [98]:
score=accuracy_score(y_test,y_pred)
print(score)

0.9650224215246637


In [16]:
df_imdb = pd.read_csv('/kaggle/input/imdb-reviews-dataset/IMDB Dataset.csv')

In [17]:
df_imdb.shape

(50000, 2)

In [18]:
df_imdb.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [21]:
corpus = []
for i in range(0, len(df_imdb)):
    review = re.sub('[^a-zA-Z]', ' ', df_imdb['review'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [22]:
corpus

['one review mention watch oz episod hook right exactli happen br br first thing struck oz brutal unflinch scene violenc set right word go trust show faint heart timid show pull punch regard drug sex violenc hardcor classic use word br br call oz nicknam given oswald maximum secur state penitentari focus mainli emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home mani aryan muslim gangsta latino christian italian irish scuffl death stare dodgi deal shadi agreement never far away br br would say main appeal show due fact goe show dare forget pretti pictur paint mainstream audienc forget charm forget romanc oz mess around first episod ever saw struck nasti surreal say readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard sold nickel inmat kill order get away well manner middl class inmat turn prison bitch due lack street skill prison experi watch oz may becom comfort uncomfort view that get touch d

In [23]:
corpus[0]

'one review mention watch oz episod hook right exactli happen br br first thing struck oz brutal unflinch scene violenc set right word go trust show faint heart timid show pull punch regard drug sex violenc hardcor classic use word br br call oz nicknam given oswald maximum secur state penitentari focus mainli emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home mani aryan muslim gangsta latino christian italian irish scuffl death stare dodgi deal shadi agreement never far away br br would say main appeal show due fact goe show dare forget pretti pictur paint mainstream audienc forget charm forget romanc oz mess around first episod ever saw struck nasti surreal say readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard sold nickel inmat kill order get away well manner middl class inmat turn prison bitch due lack street skill prison experi watch oz may becom comfort uncomfort view that get touch da

In [25]:
df_imdb['review'].loc[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [28]:
words = []
for doc in corpus:
    sents = sent_tokenize(doc)
    if not sents:
        words.append([])                # placeholder for empty/invalid doc
    else:
        for sent in sents:
            words.append(simple_preprocess(sent))

In [29]:
words

[['one',
  'review',
  'mention',
  'watch',
  'oz',
  'episod',
  'hook',
  'right',
  'exactli',
  'happen',
  'br',
  'br',
  'first',
  'thing',
  'struck',
  'oz',
  'brutal',
  'unflinch',
  'scene',
  'violenc',
  'set',
  'right',
  'word',
  'go',
  'trust',
  'show',
  'faint',
  'heart',
  'timid',
  'show',
  'pull',
  'punch',
  'regard',
  'drug',
  'sex',
  'violenc',
  'hardcor',
  'classic',
  'use',
  'word',
  'br',
  'br',
  'call',
  'oz',
  'nicknam',
  'given',
  'oswald',
  'maximum',
  'secur',
  'state',
  'penitentari',
  'focus',
  'mainli',
  'emerald',
  'citi',
  'experiment',
  'section',
  'prison',
  'cell',
  'glass',
  'front',
  'face',
  'inward',
  'privaci',
  'high',
  'agenda',
  'em',
  'citi',
  'home',
  'mani',
  'aryan',
  'muslim',
  'gangsta',
  'latino',
  'christian',
  'italian',
  'irish',
  'scuffl',
  'death',
  'stare',
  'dodgi',
  'deal',
  'shadi',
  'agreement',
  'never',
  'far',
  'away',
  'br',
  'br',
  'would',
  'say',

In [31]:
len(words)

50000

In [33]:
len(words[0]),len(words[2])

(168, 88)

In [41]:
### Lets train Word2vec from scratch
model=gensim.models.Word2Vec(words,window=5,min_count=2)

model.wv.index_to_key

['br',
 'movi',
 'film',
 'one',
 'like',
 'time',
 'good',
 'make',
 'charact',
 'see',
 'get',
 'watch',
 'even',
 'stori',
 'would',
 'realli',
 'well',
 'scene',
 'look',
 'show',
 'much',
 'end',
 'bad',
 'great',
 'peopl',
 'go',
 'love',
 'also',
 'first',
 'think',
 'act',
 'play',
 'way',
 'thing',
 'made',
 'could',
 'know',
 'say',
 'seem',
 'work',
 'plot',
 'actor',
 'two',
 'mani',
 'seen',
 'come',
 'year',
 'want',
 'take',
 'never',
 'life',
 'best',
 'tri',
 'littl',
 'ever',
 'man',
 'better',
 'give',
 'still',
 'find',
 'perform',
 'feel',
 'part',
 'use',
 'someth',
 'director',
 'actual',
 'back',
 'lot',
 'interest',
 'real',
 'guy',
 'old',
 'funni',
 'cast',
 'though',
 'live',
 'anoth',
 'music',
 'enjoy',
 'star',
 'noth',
 'role',
 'start',
 'new',
 'point',
 'set',
 'everi',
 'girl',
 'day',
 'believ',
 'world',
 'origin',
 'turn',
 'thought',
 'horror',
 'quit',
 'direct',
 'minut',
 'comedi',
 'kill',
 'us',
 'fact',
 'pretti',
 'effect',
 'action',
 'ha

In [42]:
model.corpus_count, model.epochs

(50000, 5)

In [43]:
#apply for the entire sentences
X=[]
for i in tqdm(range(len(words))):
    print("Hello",i)
    X.append(avg_word2vec(words[i]))

  0%|          | 21/50000 [00:00<04:12, 197.96it/s]

Hello 0
Hello 1
Hello 2
Hello 3
Hello 4
Hello 5
Hello 6
Hello 7
Hello 8
Hello 9
Hello 10
Hello 11
Hello 12
Hello 13
Hello 14
Hello 15
Hello 16
Hello 17
Hello 18
Hello 19
Hello 20
Hello 21
Hello 22
Hello 23
Hello 24
Hello 25
Hello 26
Hello 27
Hello 28
Hello 29
Hello 30
Hello 31
Hello 32
Hello 33


  0%|          | 57/50000 [00:00<05:29, 151.47it/s]

Hello 34
Hello 35
Hello 36
Hello 37
Hello 38
Hello 39
Hello 40
Hello 41
Hello 42
Hello 43
Hello 44
Hello 45
Hello 46
Hello 47
Hello 48
Hello 49
Hello 50
Hello 51
Hello 52
Hello 53
Hello 54
Hello 55
Hello 56
Hello 57
Hello 58
Hello 59
Hello 60
Hello 61
Hello 62
Hello 63
Hello 64
Hello 65
Hello 66


  0%|          | 89/50000 [00:00<05:50, 142.49it/s]

...

Hello 49912
Hello 49913
Hello 49914
Hello 49915
Hello 49916
Hello 49917
Hello 49918
Hello 49919
Hello 49920
Hello 49921
Hello 49922
Hello 49923
Hello 49924
Hello 49925
Hello 49926
Hello 49927
Hello 49928
Hello 49929
Hello 49930
Hello 49931
Hello 49932
Hello 49933
Hello 49934
Hello 49935
Hello 49936
Hello 49937
Hello 49938
Hello 49939
Hello 49940
Hello 49941
Hello 49942
Hello 49943


100%|█████████▉| 49963/50000 [05:36<00:00, 141.00it/s]

Hello 49944
Hello 49945
Hello 49946
Hello 49947
Hello 49948
Hello 49949
Hello 49950
Hello 49951
Hello 49952
Hello 49953
Hello 49954
Hello 49955
Hello 49956
Hello 49957
Hello 49958
Hello 49959
Hello 49960
Hello 49961
Hello 49962
Hello 49963
Hello 49964
Hello 49965
Hello 49966
Hello 49967
Hello 49968
Hello 49969
Hello 49970
Hello 49971
Hello 49972


100%|██████████| 50000/50000 [05:36<00:00, 148.49it/s]

Hello 49973
Hello 49974
Hello 49975
Hello 49976
Hello 49977
Hello 49978
Hello 49979
Hello 49980
Hello 49981
Hello 49982
Hello 49983
Hello 49984
Hello 49985
Hello 49986
Hello 49987
Hello 49988
Hello 49989
Hello 49990
Hello 49991
Hello 49992
Hello 49993
Hello 49994
Hello 49995
Hello 49996
Hello 49997
Hello 49998
Hello 49999


In [47]:
type(X)

list

In [48]:
X[0].shape

(100,)

In [45]:
X[0]

array([ 0.42198423,  0.30089122, -0.2733585 ,  0.03043621, -0.2958815 ,
       -0.34883738,  0.3856301 ,  0.22722621, -0.39937836,  0.2618341 ,
        0.21035363,  0.26372844, -0.32409167,  0.38905454, -0.3273396 ,
        0.08465467,  0.3718429 ,  0.09131666, -0.298809  , -0.8664414 ,
        0.15159783,  0.28091246,  0.23747055,  0.15457633, -0.02458489,
       -0.20798397, -0.2540738 , -0.28329563,  0.39832455, -0.06792935,
       -0.46114516, -0.17392461,  0.51106846, -0.17892753,  0.09715804,
        0.18973957,  0.08397839, -0.30994788,  0.23342913, -0.12677066,
       -0.11990678, -0.36157978, -0.29493582,  0.34675685,  0.05221816,
        0.3561322 , -0.47461727, -0.10344337, -0.17973253,  0.40229288,
       -0.17745188, -0.03546331,  0.16011786, -0.18141495,  0.32697147,
        0.25873217, -0.23073332,  0.07474116,  0.05448382,  0.01937493,
        0.1286054 , -0.63562983, -0.02138306, -0.18458763,  0.25588465,
        0.3626668 ,  0.31929109,  0.37096438,  0.1179741 , -0.11

In [49]:
X_new = (np.array(X, dtype= object))

In [52]:
y=pd.get_dummies(df_imdb['sentiment'])
y=y.iloc[:,1].values

In [55]:
y

array([ True,  True,  True, ..., False, False, False])

In [56]:
# Train Test Split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [57]:
from sklearn.ensemble import RandomForestClassifier
senitment_prediction_model = RandomForestClassifier(
    n_estimators=100, 
    max_depth=None,       
    random_state=42,      
    n_jobs=-1             
).fit(X_train, y_train)

In [60]:
#prediction
y_pred=senitment_prediction_model.predict(X_test)

In [63]:
score=accuracy_score(y_test,y_pred)
print(score)

0.8346
